<a href="https://colab.research.google.com/github/henricobela/checkpoint1_Cyber2022/blob/main/Checkpoint_Cyber_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sejam bem-vindos a minha API para INTERPOL!
####Meu Nome é Henrico, sou aluno da FIAP (RM95985)
--------------------------------------------------------------------------------
#Vamos começar?

####Esta aplicação trata-se de uma pagina na web criada com streamlit, utilizando a linguagem python, fuzzywuzzy, ngrok e regex. Ela faz uma verificação na base de dados da Interpol (fictícia), e uma verificação na base de dados de passaportes proibidos da França (fictícia). Com essa verificação é definida a entrada ou não de uma certa pessoa á França!
--------------------------------------------------------------------------------

#####Para começar

#####Primeiro vamos instalar todos os recursos necessários para a realização da API, instalando FuzzyWuzzy, Ngrok, Streamlit/ace.

--------------------------------------------------------------------------------


#1º Instalando recursos

In [1]:
!pip install -q streamlit

     |████████████████████████████████| 10.1 MB 6.4 MB/s 
     |████████████████████████████████| 164 kB 45.5 MB/s 
     |████████████████████████████████| 4.3 MB 49.6 MB/s 
     |████████████████████████████████| 181 kB 45.7 MB/s 
     |████████████████████████████████| 111 kB 37.3 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 131 kB 46.0 MB/s 
     |████████████████████████████████| 793 kB 43.3 MB/s 
     |████████████████████████████████| 428 kB 39.7 MB/s 
     |████████████████████████████████| 130 kB 49.9 MB/s 
     |████████████████████████████████| 381 kB 59.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

In [2]:
!pip install -q streamlit-ace

     |████████████████████████████████| 3.6 MB 8.8 MB/s 


In [3]:
!pip install -q fuzzywuzzy[speedup]

     |████████████████████████████████| 50 kB 3.9 MB/s 


In [4]:
!pip install -q pyngrok

     |████████████████████████████████| 745 kB 7.7 MB/s 


##2º - Criando aplicativo do streamlit para gerar a pagina web


In [5]:
!streamlit run app.py &>/dev/null&

##3º - Importando a biblioteca NGROK

In [6]:
from pyngrok import ngrok

#4º - Aplicação

In [46]:
#criando o arquivo de aplicação .py
%%writefile app.py  
import streamlit as st            #importando streamlit e setando como st
import re                         #importando o regex
from fuzzywuzzy import process    #importando process do fuzzy
import json                       #importando json
import requests                   #importando requests


def main():                                                                       #define a função que cria webpage
  st.set_page_config(page_title="France Airport", layout="centered")              #função do streamlit que configura a pagina da web, com as cartacteristicas de titulo e layout
  st.title("Imigrate Verification")                                               #função do streamlit que gera um título na pagina da web e da um nome a ele
  form = st.form(key="form")                                                      #função do streamlit que gera um formulario

  with form:                                                                      #função para gerar e abrir um formulario na tela da pagina web
    ownerName = st.text_input("Nome do Indíviduo:")                               #guarda na variavel ownerName a função do streamlit que gera um campo para se digitar, no caso, o nome do individuo
    passportNumber = st.text_input("Número do Passaporte:")                       #guarda na variavel passportNumber a função do streamlit que gera um campo para se digitar, caso o numero de passaporte
    #passportNumber = re.sub("\D", "", passportNumber)                             #função do streamlit que substitui todos caracteres por nada*********ainda testando**********
    submit = st.form_submit_button(label="SEND")                                  #guarda na variavel submit a função do streamlit que gera um botão

  nameDataJson = requests.get("https://cspinheiro.github.io/interpol.json")       #consome json dos nomes da interpol e guarda na variavel nameDataJson
  fraDataJson = requests.get("https://henricobela.github.io/fra.json")            #consome json da França com numeros de passaportes e salva na variavel fraDataJson

  def verifyNamesInterpol(names):                                                 #define a função de verificação dos nomes no endpoint da interpol
    invalidNames = nameDataJson.json()["interpol"]                                #salva na variavel invalidNames tudo que tiver interpol no arquivo json
    invalid = []                                                                  #lista invalid vazia

    extraction = process.extract(names, invalidNames)                             #salva na variavel extraction, utilizando o fuzzy para extrair os nomes do endpoint

    for x in extraction:                                                          #laço for para cada elemento "x" na variavel extraction - faça:
      if x[1] >= 76:                                                              #condição if para threshold acima ou igual da porcentagem de 80
        invalid.append(x)                                                         #acrescenta na lista vazia invalid o elemento "x"
      if len(invalid) == 0:                                                       #condição if para o comprimento da lista invalid comparando se for 0
        return "Allow"                                                            #resultado da comparação se for verdadeira retorna internamente "Allow"
      return "Deny"                                                               #se o resultado da condição for falso, retorna internamente "Deny"

  def verifyPassportNmbr(passportNmbr):                                           #define a função que verifica os numeros de passaporte
    api = re.search(r"(\D{3})", passportNmbr.lower())                             #guarda na variavel api a busca pelos elementos dentro das letras digitadas pelo usuario
    passport = re.search("(?i)(\d+)", passportNmbr.lower())                       #guarda na variavel passport a busca pelos elementos dentro do numero de passaporte digitado pelo usuario 
    x = str(api[0])
    #y = int(passport[1])

    if api and passport:                                                          #condição se encontrar na api e passport - faça:
      x_api = fraDataJson.json()[x]                                               #salva na variavel dict_api tudo que tiver x letras no arquivo json
      #y_passport = fraDataJson.json()[y]
      for item in x_api:                                                          #laço for para cada item dentro da variavel dict_api - faça:
        for value in item.values():                                               #laço for para cada valor dentro dos valores do arquivo json - faça:
          if passportNmbr == str(x) + str(value) or passportNmbr == value:        #condição que compara se o numero de passaporte digitado é igual as letras + os valores que existem no arquivo json
            return "Deny"                                                         #resultado da comparação se for falsa retorna internamente "Deny"
      return "Allow"                                                              #resultado da condição for verdadeira, retorna internamente "Allow"
 
  def verifyPerson(name, passportNmbr):                                           #define a função que verifica o nome e o passaporte digitado
    person = verifyNamesInterpol(name)                                            #guarda na variavel person o nome digitado
    passport = verifyPassportNmbr(passportNmbr)                                   #guarda na variavel passport o passaporte digitado

    if passport == "Deny" or person == "Deny":                                    #condição que diz se o passaporte for negado ou o nome da pessoa for negado no pais - faça:
      st.error("Você não pode entrar no país")                                    #função do streamlit que gera uma mensagem de erro
      return                                                                      #retorna a função do streamlit
    st.success("Voce pode entrar no país")                                        #função do streamlit que gera uma mensagem de sucesso
    return                                                                        #retorna a função do streamlit

  if submit:                                                                      #condição se for apertado o botão
    if not ownerName or not passportNumber:                                       #condição se não foi digitado o nome OU o passaport no formulario
      st.warning("Digite todos os campos por favor...")                           #função do streamlit que gera uma mensagem de aviso
    else:                                                                         #então da condição submit
      verifyPerson(ownerName, passportNumber)                                     #chamada do metodo criado para a verificação

if __name__=='__main__':                                                          #condição para inicializar o programa
  main()                                                                          #chamada do programa.

Overwriting app.py


In [47]:
!streamlit run app.py & npx localtunnel --port 8501                               #chamada para criação da pagina web e inicialização do app.py

2022-05-01 23:01:50.076 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.97.81:8501

npx: installed 22 in 3.196s
your url is: https://giant-parrots-eat-34-125-97-81.loca.lt
  Stopping...
^C
